# 02. 振幅エンベロープの可視化

音声の振幅エンベロープ（音量の変化）を可視化して、音の大きさの時間変化を理解します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. ADSR エンベロープの作成

In [ ]:
def create_adsr_envelope(attack_time, decay_time, sustain_level, release_time, 
                        total_duration, sample_rate):
    """
    ADSR エンベロープを作成
    Attack-Decay-Sustain-Release
    """
    total_samples = int(total_duration * sample_rate)
    attack_samples = int(attack_time * sample_rate)
    decay_samples = int(decay_time * sample_rate)
    release_samples = int(release_time * sample_rate)
    sustain_samples = total_samples - attack_samples - decay_samples - release_samples
    
    envelope = np.zeros(total_samples)
    
    # Attack phase
    envelope[:attack_samples] = np.linspace(0, 1, attack_samples)
    
    # Decay phase
    start_idx = attack_samples
    end_idx = start_idx + decay_samples
    envelope[start_idx:end_idx] = np.linspace(1, sustain_level, decay_samples)
    
    # Sustain phase
    start_idx = end_idx
    end_idx = start_idx + sustain_samples
    envelope[start_idx:end_idx] = sustain_level
    
    # Release phase
    start_idx = end_idx
    envelope[start_idx:] = np.linspace(sustain_level, 0, release_samples)
    
    return envelope

# パラメータ設定
sample_rate = 44100
duration = 3.0
frequency = 440

# 時間軸
t = np.linspace(0, duration, int(sample_rate * duration), False)

# ADSRエンベロープの作成
envelope = create_adsr_envelope(
    attack_time=0.1,    # アタック: 0.1秒
    decay_time=0.3,     # ディケイ: 0.3秒
    sustain_level=0.6,  # サスティン: 60%
    release_time=0.8,   # リリース: 0.8秒
    total_duration=duration,
    sample_rate=sample_rate
)

# 基本波形の生成
base_wave = np.sin(2 * np.pi * frequency * t)

# エンベロープを適用
modulated_wave = base_wave * envelope

# 可視化
plt.figure(figsize=(15, 10))

# エンベロープ
plt.subplot(3, 1, 1)
plt.plot(t, envelope, 'g-', linewidth=2)
plt.title('ADSR エンベロープ')
plt.ylabel('振幅')
plt.grid(True)
plt.axvline(x=0.1, color='r', linestyle='--', alpha=0.7, label='Attack終了')
plt.axvline(x=0.4, color='orange', linestyle='--', alpha=0.7, label='Decay終了')
plt.axvline(x=2.2, color='purple', linestyle='--', alpha=0.7, label='Release開始')
plt.legend()

# 元の波形
plt.subplot(3, 1, 2)
plt.plot(t[:4000], base_wave[:4000], 'b-', alpha=0.7)
plt.title('元の440Hz サイン波')
plt.ylabel('振幅')
plt.grid(True)

# エンベロープ適用後
plt.subplot(3, 1, 3)
plt.plot(t[:4000], modulated_wave[:4000], 'r-')
plt.plot(t[:4000], envelope[:4000], 'g--', alpha=0.7, label='エンベロープ')
plt.plot(t[:4000], -envelope[:4000], 'g--', alpha=0.7)
plt.title('エンベロープ適用後の波形')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 2. 異なるエンベロープ形状の比較

In [ ]:
# 異なるエンベロープパターンの作成
duration = 2.0
t = np.linspace(0, duration, int(sample_rate * duration), False)
base_wave = np.sin(2 * np.pi * 440 * t)

# パターン1: ピアノ風（速いアタック、長いリリース）
piano_env = create_adsr_envelope(0.01, 0.1, 0.3, 1.5, duration, sample_rate)
piano_wave = base_wave * piano_env

# パターン2: オルガン風（遅いアタック、サスティン長い）
organ_env = create_adsr_envelope(0.2, 0.1, 0.8, 0.3, duration, sample_rate)
organ_wave = base_wave * organ_env

# パターン3: 打楽器風（速いアタック、速いディケイ）
perc_env = create_adsr_envelope(0.01, 0.3, 0.1, 0.5, duration, sample_rate)
perc_wave = base_wave * perc_env

plt.figure(figsize=(15, 12))

# ピアノ風
plt.subplot(3, 2, 1)
plt.plot(t, piano_env, 'b-', linewidth=2)
plt.title('ピアノ風エンベロープ')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(3, 2, 2)
plt.plot(t[:8000], piano_wave[:8000], 'b-')
plt.title('ピアノ風波形')
plt.grid(True)

# オルガン風
plt.subplot(3, 2, 3)
plt.plot(t, organ_env, 'g-', linewidth=2)
plt.title('オルガン風エンベロープ')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(3, 2, 4)
plt.plot(t[:8000], organ_wave[:8000], 'g-')
plt.title('オルガン風波形')
plt.grid(True)

# 打楽器風
plt.subplot(3, 2, 5)
plt.plot(t, perc_env, 'r-', linewidth=2)
plt.title('打楽器風エンベロープ')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(3, 2, 6)
plt.plot(t[:8000], perc_wave[:8000], 'r-')
plt.title('打楽器風波形')
plt.xlabel('時間 (秒)')
plt.grid(True)

plt.tight_layout()
plt.show()

## 3. 音声の振幅エンベロープ抽出

In [ ]:
# 複雑な音声信号の生成（音楽のような変化を模擬）
def create_musical_phrase(duration, sample_rate):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 複数の音符を模擬
    phrase = np.zeros_like(t)
    
    # 音符1: C (261.63 Hz)
    note1_start, note1_end = 0, 0.5
    mask1 = (t >= note1_start) & (t < note1_end)
    env1 = np.exp(-(t[mask1] - note1_start) * 3)  # 指数減衰
    phrase[mask1] = env1 * np.sin(2 * np.pi * 261.63 * t[mask1])
    
    # 音符2: E (329.63 Hz)
    note2_start, note2_end = 0.5, 1.0
    mask2 = (t >= note2_start) & (t < note2_end)
    env2 = np.exp(-(t[mask2] - note2_start) * 2)
    phrase[mask2] = env2 * np.sin(2 * np.pi * 329.63 * t[mask2])
    
    # 音符3: G (392.00 Hz)
    note3_start, note3_end = 1.0, 1.8
    mask3 = (t >= note3_start) & (t < note3_end)
    env3 = np.exp(-(t[mask3] - note3_start) * 1.5)
    phrase[mask3] = env3 * np.sin(2 * np.pi * 392.00 * t[mask3])
    
    return t, phrase

# 音楽フレーズの生成
t_music, music_signal = create_musical_phrase(2.0, sample_rate)

# エンベロープの抽出（ヒルベルト変換を使用）
analytic_signal = signal.hilbert(music_signal)
amplitude_envelope = np.abs(analytic_signal)

# 移動平均によるスムージング
window_size = 1000
smoothed_envelope = np.convolve(amplitude_envelope, 
                               np.ones(window_size)/window_size, 
                               mode='same')

plt.figure(figsize=(15, 10))

# 原信号
plt.subplot(3, 1, 1)
plt.plot(t_music, music_signal, 'b-', alpha=0.7)
plt.title('模擬音楽信号')
plt.ylabel('振幅')
plt.grid(True)

# エンベロープ抽出結果
plt.subplot(3, 1, 2)
plt.plot(t_music, music_signal, 'b-', alpha=0.3, label='原信号')
plt.plot(t_music, amplitude_envelope, 'r-', linewidth=2, label='抽出エンベロープ')
plt.plot(t_music, -amplitude_envelope, 'r-', linewidth=2)
plt.title('振幅エンベロープの抽出')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# スムージング済みエンベロープ
plt.subplot(3, 1, 3)
plt.plot(t_music, amplitude_envelope, 'r-', alpha=0.5, label='原エンベロープ')
plt.plot(t_music, smoothed_envelope, 'g-', linewidth=2, label='スムージング済み')
plt.title('スムージングされたエンベロープ')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 練習問題

1. 自分だけのADSRエンベロープパラメータを設定して、新しい楽器の音を作ってみましょう
2. トレモロ効果（振幅の周期的変化）をエンベロープに追加してみましょう
3. 複数の音符を組み合わせた短いメロディーを作成し、そのエンベロープを抽出してみましょう